In [1]:
import sympy
from sympy import Rational
import numpy as np
import pandas as pd
import math
from gurobipy import *

# Read the data
train_info = pd.read_csv('train_info_single_class_multi_origin0619.csv')

# Input parameters
# Initial values of preference parameters
alpha_value_initial = 0.01
beta_value_initial = 0.01
gamma_value_initial = 0.01
g0_value_initial = 0.01

s = 0.0002 # Neighborhood radius value for sensitivity analysis

di = len(train_info['TYPE']) #Number of train services
station_interval = list(range(151019789, 151019778, -1)) # Codes for sequential train running intervels from Tuqiao station to Sihui Dong station
origins_input = [151019789, 151019787, 151019785, 151019784] # Station codes of multiple origins
origins_interval_num = {151019789:11, 151019787:9, 151019785:7, 151019784:6} # Number of train running intervels of multiple origins
userclass_input = {origins_input[0]:[0], origins_input[1]:[0], origins_input[2]:[0], origins_input[3]:[0]} # Number of commuter class of multiple origins（single class）

upper_parameters_record = sympy.Matrix([alpha_value_initial, beta_value_initial, gamma_value_initial, g0_value_initial]) # Upper level decision variable value record

upper_obj_record = [] # Upper objective value record
upper_obj_sensitivity_record = [] # Sensitivity analysis-based upper objective value record
indicator_record = [] # Convergence criterion value record (δ)

# Train choice sets for single class commuters from different stations
train_choice_set_input = {}
train_choice_set_input[origins_input[0]] = {0:list(range(8,47))}
train_choice_set_input[origins_input[1]] = {0:list(range(5,47))}
train_choice_set_input[origins_input[2]] = {0:list(range(5,47))}
train_choice_set_input[origins_input[3]] = {0:list(range(3,47))}

# Travel demand for single class commuters from different stations
demand_input = {}
demand_input[origins_input[0]] = {0: 496}
demand_input[origins_input[1]] = {0: 530}
demand_input[origins_input[2]] = {0: 430}
demand_input[origins_input[3]] = {0: 551}

# Desired arrival times for single class commuters from different stations
desired_arrival_time_input = {}
desired_arrival_time_input[origins_input[0]] = {0:30900}
desired_arrival_time_input[origins_input[1]] = {0:31500}
desired_arrival_time_input[origins_input[2]] = {0:30900}
desired_arrival_time_input[origins_input[3]] = {0:31500}

# Commuters' travel times for different trains at different stations
T = {}
for o in origins_input:
    for j in train_info['TYPE']:
        T[(o,j)] = train_info[train_info['TYPE'] == j]['TRIP_TIME_average_%d'%o].iloc[0]
        
# Train running time between adjacent stations 
Tv = {151019789: 110, 151019788: 130, 151019787: 140, 151019786: 120, 151019785: 160, 151019784: 180, 151019783: 180, 151019782: 180, 151019781: 200, 151019780: 170, 151019779: 110}

# Background passenger flows along the Batong Line during the morning peak period
n0 = {}
for j in train_info['TYPE']:
    for i in station_interval:
        n0[(j,i)] = train_info[train_info['TYPE'] == j]['%d'%i].values[0]

# Schedule delay time arriving early and Schedule delay time arriving late for single class commuters from different stations
TE = {}
TL = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            TE[(o,i,j)] = max(desired_arrival_time_input[o][i] - train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0], 0)
            TL[(o,i,j)] = max(train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0] - desired_arrival_time_input[o][i], 0)

In [2]:
# Definition of symbols for matrix calculation
# Symbols representing preference parameters
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
g0 = sympy.Symbol('g0')

lamb = sympy.Symbol('lamb')
no_num = sympy.Symbol('no_num')

# Symbols for departure rates
nis = {}
nis_keys = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis[(o,i,j)] = sympy.Symbol('n%d,%d,%d'%(o,i,j))
            nis_keys.append((o,i,j))
            
# Symbols for lagrange multiplier associated with constraints (10b)
miu = {}
for o in origins_input:
    for i in userclass_input[o]:
        miu[(o,i)] = sympy.Symbol('u%d,%d'%(o,i))

# Observed departure rates
y_observed_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            y_observed_li.append(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)][j-1])
y_observed = sympy.Matrix(y_observed_li)        
        
y_observed_dic = {}
for o in origins_input:
    y_observed_dic[o] = {}
    for i in userclass_input[o]:
        y_observed_dic[o][i] = sympy.Matrix(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)])

# Values of preference parameters
upper_parameters_value = {}
upper_parameters_value[alpha] = alpha_value_initial
upper_parameters_value[beta] = beta_value_initial
upper_parameters_value[gamma] = gamma_value_initial
upper_parameters_value[g0] = g0_value_initial

In [3]:
# In-vehicle commuter flow aggregations
flow_nis_aggregate = sympy.zeros(di,len(station_interval))
flow_nis_aggregate_dic = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] = flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(nis[(o,i,j)], (1, origins_interval_num[o])))

for j in train_info['TYPE']:
    for i in station_interval:
        flow_nis_aggregate_dic[(j, i)] = flow_nis_aggregate[(j - 1),(151019789 - i)]  

# In-vehicle crowding cost calculation
def invehicle_crowding_cost(origin, train, Tv_dic, n0_dic, flow_dic, g0_para):
    crowding_cost = 0
    for i in range(origin, 151019778, -1):
        crowding_cost = crowding_cost + g0_para * Tv_dic[i] * (flow_dic[(train, i)] + n0_dic[(train, i)] - 256) / 275
    return crowding_cost

In [4]:
# Derivatives of lagrange functions with respect to n
Ln = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            Ln.append((sympy.log(nis[(o,i,j)]) + 1) + invehicle_crowding_cost(o, j, Tv, n0, flow_nis_aggregate_dic, g0) + alpha * T[(o,j)] + beta * TE[(o,i,j)] + gamma * TL[(o,i,j)] - miu[(o,i)])            
            
Noi = []
for o in origins_input:
    for i in userclass_input[o]:
        noi = 0
        for j in train_choice_set_input[o][i]:
            noi = noi + nis[(o,i,j)]
        Noi.append(noi) 

# Block matrix B11
A_li = []
for num in range(0,len(Ln)):
    ln = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                ln.append(sympy.diff(Ln[num],nis[(o,i,j)]))
    A_li.append(ln)
    
A = sympy.Matrix(A_li)

# Block matrix B12
B_li = []
for num in range(0,len(Ln)):
    lu = []
    for o in origins_input:
        for i in userclass_input[o]:
            lu.append(sympy.diff(Ln[num],miu[(o,i)]))
    B_li.append(lu)
    
B = sympy.Matrix(B_li)

# Block matrix B21
C_li = []
for num in range(0,len(Noi)):
    nn = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nn.append(sympy.diff(Noi[num],nis[(o,i,j)]))
    C_li.append(nn)  
    
C = sympy.Matrix(C_li)

# Block matrix B22
D_li = []
for num in range(0,len(Noi)):
    nu = []
    for o in origins_input:
        for i in userclass_input[o]:
            nu.append(sympy.diff(Noi[num],miu[(o,i)]))
    D_li.append(nu)        

D = sympy.Matrix(D_li) 

# Block matrix N
N_li = []
for num in range(0,len(Ln)):
    lp = []
    lp.append(sympy.diff(Ln[num],alpha))
    lp.append(sympy.diff(Ln[num],beta))
    lp.append(sympy.diff(Ln[num],gamma))
    lp.append(sympy.diff(Ln[num],g0))
    N_li.append(lp)
    
N = sympy.Matrix(N_li)

epoch = 0
indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))

In [5]:
# Generalized travel cost of train services for single class commuters
def train_cost(flow_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_dic = {}
    for o in origins_input:
        for i in userclass[o]:
            for j in train_choice_set[o][i]:           
                train_cost_dic[(o, i, j)] = invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]

    return train_cost_dic

# Logit-based commuting flow loading
def logit(origin, user, train_cost_dic, demand, choice_set):
    
    cost_exp_li = []
    for j in train_info['TYPE']:
        if j in choice_set:
            cost_exp_li.append(sympy.exp( - train_cost_dic[(origin, user, j)]))
        else:
            cost_exp_li.append(0)
            
    flow = (sympy.Matrix(cost_exp_li) / np.sum(cost_exp_li)) * demand 
    return flow

# Logit-based commuting flow loading of single class commuters
def multiorigin_multiclass_logit(userclass, train_cost_dic, demand_dic, train_choice_set):
    
    flow_multiorigin_multiclass_aggregate = sympy.zeros(di,len(station_interval))
    flow_multiorigin_multiclass_dic = {}
    flow_multiorigin_multiclass_aggregate_dic = {}
    
    for o in origins_input:
        flow_multiorigin_multiclass_dic[o] = {}
        for i in userclass[o]:
            flow_multiorigin_multiclass_dic[o][i] = logit(o, i, train_cost_dic, demand_dic[o][i], train_choice_set[o][i])
            flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] = flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multiorigin_multiclass_dic[o][i], (1, origins_interval_num[o])))
    
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_multiorigin_multiclass_aggregate_dic[(j, i)] = flow_multiorigin_multiclass_aggregate[(j - 1),(151019789 - i)]
    
    return flow_multiorigin_multiclass_aggregate, flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic

# Generalized stochastic travel cost of train services for single class commuters
def train_cost_stochastic(flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_stochastic_dic = {}
    for o in origins_input:
        train_cost_stochastic_dic[o] = {}
        for i in userclass[o]:
            flow_origin_class_dic = dict(zip(train_info['TYPE'], flow_multiorigin_multiclass_dic[o][i]))
            train_cost_stochastic_dic[o][i] = {}
            for j in train_info['TYPE']:  
                if j in train_choice_set[o][i]:
                    train_cost_stochastic_dic[o][i][j] = (sympy.log(flow_origin_class_dic[j])+ 1) + invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_multiorigin_multiclass_aggregate_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]
                else:
                    train_cost_stochastic_dic[o][i][j] = no_num
                    
    return train_cost_stochastic_dic

# Bisection method to determine the step size
def binary_search(equation):
    a = 0
    b = 1
    fa = equation.evalf(subs = {lamb:a})
    fb = equation.evalf(subs = {lamb:b})
    while a <= b:
        mid = (a + b) / 2
        fm = equation.evalf(subs = {lamb:mid})
        if abs(fm) < 0.1:
            #print("方程的根为",mid)
            break
        if fa * fm < 0:
            a = a
            b = mid
            fb = equation.evalf(subs = {lamb:b})
            #print("解在[",a,",",b,"]之间")
        elif fb * fm < 0:
            a = mid
            b = b
            fa = equation.evalf(subs = {lamb:a})
            #print("解在[",a,",",b,"]之间")
            
    return mid

In [6]:
# Solve the rail corridor commuting equilibrium model(convex combination iteration algorithm)
def train_flow_assignment(userclass, train_choice_set, multi_demand, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    n = 0
    indicator_am = 1
    flow_dic_initial = {}
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_dic_initial[(j, i)] = 0
    cost_dic = train_cost(flow_dic_initial, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    flow_aggregate_x_new, flow_multi_dic_x_new, flow_aggregate_dic_x_new = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
    
    while (indicator_am > 0.0001) or (n <= 20):
    
        flow_multi_dic_x = flow_multi_dic_x_new
        flow_aggregate_dic_x = flow_aggregate_dic_x_new

        # Train costs update
        cost_dic = train_cost(flow_aggregate_dic_x, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        # Logit-based commuting flow loading
        flow_aggregate_y, flow_multi_dic_y, flow_aggregate_dic_y = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
        
        # Auxiliary flow calculation
        flow_aggregate_aux = sympy.zeros(di,len(station_interval))
        flow_multi_dic_aux = {}
        flow_aggregate_dic_aux = {}
        
        for o in origins_input:
            flow_multi_dic_aux[o] = {}
            for i in userclass[o]:
                flow_multi_dic_aux[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb
                flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] = flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_aux[o][i], (1, origins_interval_num[o])))

        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_aux[(j, i)] = flow_aggregate_aux[(j - 1),(151019789 - i)]
                
        # Calculate the step size in this iteration
        train_cost_stochastic_aux_dic = train_cost_stochastic(flow_multi_dic_aux, flow_aggregate_dic_aux, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        equation_lamb = 0
        for o in origins_input:
            for i in userclass[o]:
                equation_lamb = equation_lamb + ((flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]).T * sympy.Matrix(list(train_cost_stochastic_aux_dic[o][i].values())))[0,0]

        lamb_value_df = binary_search(equation_lamb)
    
        # Determine the new iteration starting point
        flow_multi_dic_x_new = {}
        flow_aggregate_dic_x_new = {}
        flow_aggregate_x_new = sympy.zeros(di,len(station_interval))
        for o in origins_input:
            flow_multi_dic_x_new[o] = {}
            for i in userclass[o]:
                flow_multi_dic_x_new[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb_value_df
                flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] = flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_x_new[o][i], (1, origins_interval_num[o])))
    
        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_x_new[(j, i)] = flow_aggregate_x_new[(j - 1),(151019789 - i)]
        
        # Convergence criterion
        numerator = 0
        denominator = 0
        for o in origins_input:
            for i in userclass[o]:
                numerator = numerator + ((flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]).T * (flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]))[0,0]
                denominator = denominator + (sympy.ones(1,di) * flow_multi_dic_x[o][i])[0,0]
        
        indicator_am = (numerator ** 0.5) / denominator
        n = n + 1
        print(indicator_am)
        print(n)
        if (indicator_am <= 0.00001) or (n >= 20):
            break
    print(indicator_am)
    print(n)
    
    cost_dic_x_new = train_cost(flow_aggregate_dic_x_new, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    
    return flow_multi_dic_x_new, flow_aggregate_x_new, flow_aggregate_dic_x_new, cost_dic_x_new

In [ ]:
while (indicator >= 0.00015) or (epoch == 0):
    
    epoch = epoch + 1
    flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0])
    
    # Departure rates of single class commuters of multiple origins
    nis_value_dic = {}
    nis_value_li = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
                nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         
    nis_value_matrix = sympy.Matrix(nis_value_li)
    
    # Upper level objective value record
    upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
    print(f'Upper level objective value-equilibrium:{upper_obj_record[-1]}')
    
    # Set of values of symbols
    matrix_parameters_value_all = {}
    matrix_parameters_value_all.update(nis_value_dic)
    matrix_parameters_value_all.update(upper_parameters_value)
    
    # Values of matrixes
    A_value = np.matrix(A.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    B_value = np.matrix(B.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    C_value = np.matrix(C.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    D_value = np.matrix(D.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    N_value = np.matrix(N.evalf(subs = matrix_parameters_value_all)).astype(np.float64)

    # Gradient calculation
    I11_value = A_value.I + A_value.I * B_value * (D_value - C_value * A_value.I * B_value).I * C_value * A_value.I
    Grad = - I11_value * N_value    
    
    # Reaction function (equation (20))
    x = sympy.Matrix([alpha, beta, gamma, g0])
    x0 = sympy.Matrix([matrix_parameters_value_all[alpha], matrix_parameters_value_all[beta], matrix_parameters_value_all[gamma], matrix_parameters_value_all[g0]])
    y = nis_value_matrix + Grad * (x - x0)
    
    # Convert to matrix form. lhs is the coefficient matrix，rhs are the right hand terms
    lhs, rhs = sympy.linear_eq_to_matrix(y - y_observed, [alpha, beta, gamma, g0])
    lh = np.array(lhs).astype(np.float64)
    rh = np.array(rhs).astype(np.float64)
    
    # Solving the upper level model
    upper_model = Model("Bilevel upper model")
    upper_model.Params.NonConvex = 2
    
    # Decision variables definition of upper level model
    alpha_up = upper_model.addVar(lb = upper_parameters_value[alpha] - s, ub = upper_parameters_value[alpha] + s, vtype = GRB.CONTINUOUS, name = "alpha_up")
    beta_up = upper_model.addVar(lb = upper_parameters_value[beta] - s, ub = upper_parameters_value[beta] + s, vtype = GRB.CONTINUOUS, name = "beta_up")
    gamma_up = upper_model.addVar(lb = upper_parameters_value[gamma] - s, ub = upper_parameters_value[gamma] + s, vtype = GRB.CONTINUOUS, name = "gamma_up")
    g0_up = upper_model.addVar(lb = upper_parameters_value[g0] - s, ub = upper_parameters_value[g0] + s, vtype = GRB.CONTINUOUS, name = "g0_up")

    # Objective function of upper level model
    upper_model.setObjective(quicksum((lh[i,0] * alpha_up + lh[i,1] * beta_up + lh[i,2] * gamma_up + lh[i,3] * g0_up - rh[i,0])**2 for i in range(lh.shape[0])) ,GRB.MINIMIZE)
    
    # Constraints of upper level model
    upper_model.addConstr( alpha_up >= 0)
    upper_model.addConstr( beta_up >= 0)
    upper_model.addConstr( gamma_up >= 0)
    upper_model.addConstr( g0_up >= 0)
    
    upper_model.optimize()    
    
    # Optimization value of objective function of upper level model
    upper_model_objvalue = upper_model.ObjVal
    
    # Optimization values of decision variables of upper level model
    upper_parameters_value = {}
    upper_parameters_value[alpha] = [v.x for v in upper_model.getVars() if v.varName == 'alpha_up'][0]
    upper_parameters_value[beta] = [v.x for v in upper_model.getVars() if v.varName == 'beta_up'][0]
    upper_parameters_value[gamma] = [v.x for v in upper_model.getVars() if v.varName == 'gamma_up'][0]
    upper_parameters_value[g0] = [v.x for v in upper_model.getVars() if v.varName == 'g0_up'][0]
    
    # Calculate the indicator of convergence criterionc of upper level model
    upper_parameters_record = upper_parameters_record.col_insert(upper_parameters_record.shape[1], sympy.Matrix([upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0]]))
    indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))
    indicator_record.append(indicator)
    upper_obj_sensitivity_record.append(upper_model_objvalue)
    print(f'Upper level objective value-sensitivity analysis:{upper_model_objvalue}')
    print(f'indicator:{indicator}')
    print(f'Parameter optimization result：{upper_parameters_value}')
    print(f'Epoch:{epoch}')